In [2]:
import re
import json
import csv
import random
import requests
import pprint
import pickle
import unidecode
import numpy as np
import pandas as pd
from random import sample
import reverse_geocode
import seaborn as sns
from sklearn import metrics
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import chisquare
from collections import Counter, defaultdict
from sklearn.metrics import cohen_kappa_score

In [79]:
iop_clean_name_self_gender_ford = pd.read_csv('/home/haopeng/Data/Promotion/iop_clean_name_self_gender_ford.csv')

In [80]:
iop_clean_name_self_gender_ford.head()

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
0,Satish Kumar,India,Male,Male
1,Priya Sachin Jadhav,India,Female,Female
2,Arunkumar Bongale,India,Male,Male
3,Arunkumar Bongale,India,Male,Male
4,Arunkumar Bongale,India,Male,Male


In [81]:
iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford.gender_ford == 'invalid']

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
120,G W (Guo Wei) Yang,China,Male,invalid
349,C Zhang,China,Male,invalid
552,E C Marino,Brazil,Male,invalid
553,E C Marino,Brazil,Male,invalid
554,E C Marino,Brazil,Male,invalid
...,...,...,...,...
682180,V A Anvar,NaN,Male,invalid
682181,V A Anvar,NaN,Male,invalid
682182,V A Anvar,NaN,Male,invalid
682183,V A Anvar,NaN,Male,invalid


In [82]:
iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford.gender_ford == 'Unisex']

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
19,Jia Zhou,China,Male,Unisex
20,Jia Zhou,China,Male,Unisex
21,Jia Zhou,China,Male,Unisex
22,Jia Zhou,China,Male,Unisex
39,Young Hee Lee,Korea (Republic of),Male,Unisex
...,...,...,...,...
682248,Xing Huang,China,Male,Unisex
682258,Yu Chen,China,Male,Unisex
682259,Yu Chen,China,Male,Unisex
682260,Yu Chen,China,Male,Unisex


In [83]:
iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford.gender_ford == 'Unknown']

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
35,Yuanlie Yu,Saudi Arabia,Male,Unknown
36,Yuanlie Yu,Saudi Arabia,Male,Unknown
37,Yuanlie Yu,Saudi Arabia,Male,Unknown
38,Shu-feng Zhang,China,Male,Unknown
42,Aming Xie,China,Male,Unknown
...,...,...,...,...
682243,Chi-Te Liang,Taiwan,Male,Unknown
682244,Chi-Te Liang,Taiwan,Male,Unknown
682245,Guo-Hua Zhong,China,Male,Unknown
682246,Guo-Hua Zhong,China,Male,Unknown


In [63]:
iop_clean_name_self_gender_ford = iop_clean_name_self_gender_ford.loc[~iop_clean_name_self_gender_ford['gender_ford'].isin(['invalid', 'Unisex', 'Unknown'])]
iop_clean_name_self_gender_ford.index = range(len(iop_clean_name_self_gender_ford))


In [64]:
iop_clean_name_self_gender_ford['Contact Confirmed Gender'].value_counts()

Male                 350969
Female                81919
Prefer not to say      9660
Non-binary              388
Name: Contact Confirmed Gender, dtype: int64

In [84]:
iop_clean_name_self_gender_ford = iop_clean_name_self_gender_ford.loc[~iop_clean_name_self_gender_ford['Contact Confirmed Gender'].isin(['Non-binary', 'Prefer not to say'])]
iop_clean_name_self_gender_ford.index = range(len(iop_clean_name_self_gender_ford))


In [66]:
iop_clean_name_self_gender_ford['Contact Confirmed Gender'].value_counts()

Male      350969
Female     81919
Name: Contact Confirmed Gender, dtype: int64

In [67]:
iop_clean_name_self_gender_ford['gender_ford'].value_counts()

Male      329681
Female    103207
Name: gender_ford, dtype: int64

In [68]:
len(iop_clean_name_self_gender_ford)

432888

In [85]:
# remove 'Non-binary', 'Prefer not to say'
print(metrics.classification_report(iop_clean_name_self_gender_ford['Contact Confirmed Gender'], iop_clean_name_self_gender_ford['gender_ford']))


/home/haopeng/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/haopeng/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Female       0.74      0.65      0.69    118303
        Male       0.98      0.59      0.74    550477
      Unisex       0.00      0.00      0.00         0
     Unknown       0.00      0.00      0.00         0
     invalid       0.00      0.00      0.00         0

    accuracy                           0.60    668780
   macro avg       0.35      0.25      0.29    668780
weighted avg       0.94      0.60      0.73    668780



/home/haopeng/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
# remove 'invalid', 'Unisex', 'Unknown'
# remove 'Non-binary', 'Prefer not to say'
print(metrics.classification_report(iop_clean_name_self_gender_ford['Contact Confirmed Gender'], iop_clean_name_self_gender_ford['gender_ford']))


              precision    recall  f1-score   support

      Female       0.74      0.93      0.83     81919
        Male       0.98      0.92      0.95    350969

    accuracy                           0.93    432888
   macro avg       0.86      0.93      0.89    432888
weighted avg       0.94      0.93      0.93    432888



In [70]:
iop_clean_name_self_gender_ford.loc[(iop_clean_name_self_gender_ford['Contact Confirmed Gender'] == 'Male') & \
                                    (iop_clean_name_self_gender_ford['gender_ford'] == 'Female')]

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
6,Saroj Prasad Dash,Sweden,Male,Female
36,Xiaoxu Zhao,Singapore,Male,Female
37,Jinghua Teng,Singapore,Male,Female
47,Ping Chen,China,Male,Female
60,Fengyun Wang,China,Male,Female
...,...,...,...,...
432876,Yanwei Ma,China,Male,Female
432879,Feifei Niu,China,Male,Female
432880,Feifei Niu,China,Male,Female
432881,Feifei Niu,China,Male,Female


In [71]:
iop_clean_name_self_gender_ford.loc[(iop_clean_name_self_gender_ford['Contact Confirmed Gender'] == 'Female') & \
                                    (iop_clean_name_self_gender_ford['gender_ford'] == 'Male')]

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
8,Dhavala Suri,India,Female,Male
11,Dhavala Suri,India,Female,Male
131,Yuanhong Xu,China,Female,Male
132,Yuanhong Xu,China,Female,Male
133,Yuanhong Xu,China,Female,Male
...,...,...,...,...
432693,Yong Li,China,Female,Male
432869,Chao Dong,China,Female,Male
432870,Chao Dong,China,Female,Male
432871,Chao Dong,China,Female,Male


In [72]:
# these names have self-repoted gender as both male and female
cn = 1
for name, sub in iop_clean_name_self_gender_ford.groupby('clean_name'):
    if len(set(sub['Contact Confirmed Gender'])) > 1:
        cn += 1
        print(name)

Anqi  Huang
Bao  Yang
Bei  Liu
Bin  Du
Bin  Huang
Bin  Liu
Bin  Wu
Bin  Xiang
Bin  Xu
Bin  Zhang
Cai  Ning
Caixia  Li
Can  Cui
Can  Wang
Ce  Liang
Chang  Chen
Chang  Liu
Chao  Dong
Chao  Han
Chao  Song
Chao  Wang
Chao  Xing
Chao  Xu
Cheng  Li
Cheng  Peng
Chenyu  Wang
Chi  Sun
Chi  Wang
Chi  Zhang
Chong  Liu
Chun  Liu
Chunyan  Li
Chunyan  Liu
Cong  Wang
Dan  Dai
Dan  Li
Dan  Liu
Dan  Luo
Dan  Sun
Dan  Wang
Dan  Zhang
Dan  Zhao
Dehua  Wang
Deng  Pan
Di  Jiang
Di  Liu
Di  Wu
Dong  Liu
Duo  Zhang
Fang  Chen
Fang  Li
Fang  Liu
Fang  Wang
Fang  Yang
Fang  Zhang
Fang  Zhou
Fangfang  Zhang
Fei  Chen
Fei  Gao
Fei  Huang
Fei  Li
Fei  Liu
Fei  Shi
Fei  Wang
Fei  Yan
Fei  Yang
Fei  Zhao
Fengyun  Wang
Gang  Yu
Guang  Song
Haiyan  Chen
Haiyan  Lu
Haiyan  Wang
Han  Li
Han  Liu
Han  Wang
Han  Zhang
Han  Zhou
Hao  Chen
Hao  Liu
Hao  Qiu
Hao  Zhang
He  Li
He  Xu
Hiromi  Takahashi
Hong  Chen
Hong  He
Hong  Li
Hong  Qi
Hong  Wang
Hong  Xu
Hong  Zhao
Hong Li
Hui  Cao
Hui  Chen
Hui  Gao
Hui  Huang
Hui  Jian

In [73]:
cn

475

In [74]:
# e.g.
iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford['clean_name'] == 'Fengyun  Wang']

,clean_name,Task Country (at time of activity),Contact Confirmed Gender,gender_ford
60,Fengyun Wang,China,Male,Female
24207,Fengyun Wang,China,Male,Female
244326,Fengyun Wang,China,Female,Female
244327,Fengyun Wang,China,Female,Female
244328,Fengyun Wang,China,Female,Female
378632,Fengyun Wang,China,Female,Female
378633,Fengyun Wang,China,Female,Female
378634,Fengyun Wang,China,Female,Female


In [75]:
iop_clean_name_self_gender_ford['Task Country (at time of activity)'].value_counts()

China                         71869
India                         47706
United States of America      43871
Iran (Islamic Republic of)    23713
Germany                       20452
                              ...  
Palau                             1
Jersey                            1
Djibouti                          1
Saint Barthélemy                  1
Namibia                           1
Name: Task Country (at time of activity), Length: 153, dtype: int64

In [76]:
# remove 'invalid', 'Unisex', 'Unknown'
# remove 'Non-binary', 'Prefer not to say'

# Names from China
tem_df = iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford['Task Country (at time of activity)'] == 'China']
print(metrics.classification_report(tem_df['Contact Confirmed Gender'], tem_df['gender_ford']))

              precision    recall  f1-score   support

      Female       0.49      0.83      0.61     17925
        Male       0.93      0.71      0.80     53944

    accuracy                           0.74     71869
   macro avg       0.71      0.77      0.71     71869
weighted avg       0.82      0.74      0.76     71869



In [77]:
# remove 'invalid', 'Unisex', 'Unknown'
# remove 'Non-binary', 'Prefer not to say'

# Names not from China
tem_df = iop_clean_name_self_gender_ford.loc[iop_clean_name_self_gender_ford['Task Country (at time of activity)'] != 'China']
print(metrics.classification_report(tem_df['Contact Confirmed Gender'], tem_df['gender_ford']))

              precision    recall  f1-score   support

      Female       0.85      0.96      0.90     63994
        Male       0.99      0.96      0.98    297025

    accuracy                           0.96    361019
   macro avg       0.92      0.96      0.94    361019
weighted avg       0.97      0.96      0.96    361019

